# Orbital rotations and quadratic Hamiltonians

In this tutorial, we show how to use ffsim to perform orbital rotations and time evolution by a quadratic Hamiltonian.

## Orbital rotations

A fundamental operation in the simulation of a system of fermionic modes is a rotation
of the orbital basis. An orbital rotation is described by an $N \times N$ unitary matrix
$\mathbf{W}$ and we denote the corresponding operator as $\mathcal{W}$. This operator has
the following action on the fermionic creation operators $\set{a^\dagger_{i,\sigma}}$:

$$
    \mathcal{W} a^\dagger_{i,\sigma} \mathcal{W}^\dagger = \sum_j \mathbf{W}_{ji} a^\dagger_{j,\sigma}.
$$

That is, $a^\dagger_{i,\sigma}$ is mapped to a new operator $b^\dagger_{i,\sigma}$ where
$b^\dagger_{i,\sigma}$ is a linear combination of the operators $\set{a^\dagger_{i,\sigma}}$
with coefficients given by the $i$-th column of $\mathbf{W}$. The $\set{b^\dagger_{i,\sigma}}$
also satisfy the fermionic anticommutation relations, so they are
creation operators in a rotated basis.
The mapping $\mathbf{W} \mapsto \mathcal{W}$ satisfies the properties

$$
\begin{align*}
    \mathbf{W}^\dagger &\mapsto \mathcal{W}^\dagger, \\
    \mathbf{W}_1 \mathbf{W}_2 &\mapsto \mathcal{W}_1 \mathcal{W}_2
\end{align*}
$$

for any unitary matrices $\mathbf{W}$, $\mathbf{W}_1$, and $\mathbf{W}_2$.

In ffsim, an orbital rotation is performed by passing the vector to be transformed and the matrix $\mathbf{W}$ to the `apply_orbital_rotation` function. For example:

In [1]:
import ffsim

# Set the number of orbitals and their occupancies
norb = 6
nelec = (3, 2)

# Create the Hartree-Fock state
vec = ffsim.hartree_fock_state(norb, nelec)

# Generate a random orbital rotation
orbital_rotation = ffsim.random.random_unitary(norb, seed=1234)

# Apply the orbital rotation to the statevector
vec = ffsim.apply_orbital_rotation(vec, orbital_rotation, norb=norb, nelec=nelec)


## Time evolution by a quadratic Hamiltonian

Orbital rotations can be used to implement time evolution by a quadratic Hamiltonian. In this section, we'll demonstrate how to use ffsim to perform this task.


A quadratic Hamiltonian is an operator of the form (here we only consider Hamiltonians
with particle number and spin symmetry)

$$
    \mathcal{M} = \sum_{ij,\sigma} \mathbf{M}_{ij} a^\dagger_{i,\sigma} a_{j,\sigma}
$$

where $\mathbf{M}$ is a Hermitian matrix. A quadratic Hamiltonian can always be rewritten as

$$
    \mathcal{M} = \mathcal{W} \left(\sum_{i,\sigma} \lambda_i n_{i,\sigma}\right)\mathcal{W}^\dagger
$$

where the $\set{\lambda_i}$ are real numbers, $\mathcal{W}$ is an orbital rotation,
and we have introduced the occupation number operator $n_{i, \sigma} = a^\dagger_{i,\sigma} a_{i,\sigma}$.
The $\set{\lambda_i}$ and the unitary matrix $\mathbf{W}$ describing the orbital rotation are obtained
from an eigendecomposition of $\mathbf{M}$:

$$
    \mathbf{M}_{ij} = \sum_k \lambda_k \mathbf{W}_{ik} \mathbf{W}_{jk}^*.
$$

For our example, we will generate a random quadratic Hamiltonian and apply time evolution to its ground state. The state should simply pick up a phase proportional to its energy. To compute the ground state, we'll convert the Hamiltonian to an instance of `scipy.sparse.linalg.LinearOperator` and use Scipy to compute the lowest eigenvalue and eigenvector.

In [2]:
import numpy as np
import scipy.sparse.linalg

# Set the number of orbitals and their occupancies
norb = 6
nelec = (3, 2)
n_alpha, n_beta = nelec
occupied_orbitals = (range(n_alpha), range(n_beta))

# Generate a random quadratic Hamiltonian
one_body_tensor = ffsim.random.random_hermitian(norb, seed=1234)

# Convert the Hamiltonian to a LinearOperator
hamiltonian = ffsim.contract.one_body_linop(one_body_tensor, norb=norb, nelec=nelec)

# Get the ground state of the Hamiltonian
eigs, vecs = scipy.sparse.linalg.eigsh(hamiltonian, which="LA", k=1)
eig = eigs[0]
vec = vecs[:, 0]

Time evolution by $\mathcal{M}$ can be implemented with the following steps:

- Compute the numbers $\set{\lambda_i}$ and the matrix $\mathbf{W}$ by performing an eigendecomposition of $\mathbf{M}$.
- Perform the orbital rotation $\mathcal{W}^\dagger$, which corresponds to the matrix $\mathbf{W}^\dagger$.
- Perform time evolution by the operator $\sum_{i,\sigma} \lambda_i n_{i,\sigma}$.
- Perform the orbital rotation $\mathcal{W}^\dagger$, which corresponds to the matrix $\mathbf{W}$.

ffsim includes a function for performing time evolution by an operator of the form $\sum_{i,\sigma} \lambda_i n_{i,\sigma}$, called `apply_num_op_sum_evolution`. In fact, this function can also perform the orbital rotation for you, but for illustrative purposes we will first perform the orbital rotation explicitly.

In [3]:
time = 1.0

# Compute the orbital rotation and energies
energies, orbital_rotation = np.linalg.eigh(one_body_tensor)

# Rotate to the basis in which the Hamiltonian is diagonal
evolved_vec = ffsim.apply_orbital_rotation(
    vec, orbital_rotation.T.conj(), norb=norb, nelec=nelec
)
# Apply time evolution
evolved_vec = ffsim.apply_num_op_sum_evolution(
    evolved_vec, energies, time=time, norb=norb, nelec=nelec
)
# Undo basis rotation
evolved_vec = ffsim.apply_orbital_rotation(
    evolved_vec, orbital_rotation, norb=norb, nelec=nelec
)

# Check the result
expected_phase = np.exp(-1j * eig * time)
np.testing.assert_allclose(evolved_vec, expected_phase * vec)

As mentioned above, the function `apply_num_op_sum_evolution` can actually apply the orbital rotation for you if you pass it as an argument, as demonstrated below. Calling the function this way may give better performance.

In [4]:
evolved_vec_alt = ffsim.apply_num_op_sum_evolution(
    vec, energies, time=time, norb=norb, nelec=nelec, orbital_rotation=orbital_rotation
)
np.testing.assert_allclose(evolved_vec_alt, evolved_vec)